In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras import layers
from keras.datasets import mnist
from keras.models import Model


In [ ]:
def preprocess(array):
    # normaliza y corrige dimensiones
    array = array.astype("float32") / 255
    array = np.reshape(array, (len(array), 28, 28, 1))

    return array


In [ ]:
def add_noise(array, factor=0.4):
    noisy_array = array + factor * np.random.normal(loc=0.0, scale=1.0, size=array.shape)

    return np.clip(noisy_array, 0.0, 1.0)

In [ ]:
def display_random(array1, array2, n_img=10):
    indices = np.random.randint(len(array1), size=n_img)
    images1 = array1[indices, :]
    images2 = array2[indices, :]

    plt.figure(figsize=(20, 4))
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(2, n_img, i + 1)
        plt.imshow(image1.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n_img, i + 1 + n_img)
        plt.imshow(image2.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

## preparacion de datos

In [ ]:
(train_data, _), (test_data, _) = mnist.load_data()

train_data = preprocess(train_data)
test_data = preprocess(test_data)

noisy_train_data = add_noise(train_data)
noisy_test_data = add_noise(test_data)

display_random(train_data, noisy_train_data)

## Autoencoder

In [ ]:
input = layers.Input(shape=(28,28,1))

# encoder
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)

# decoder
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

# autoencoder model
# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()

In [ ]:
hist = autoencoder.fit(
    x=train_data,
    y=train_data,
    epochs=50,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data),
)

In [ ]:
predictions = autoencoder.predict(test_data)
display_random(test_data, predictions)

### autoencoder para limpieza
Una vez comprobamos que el autoencoder funciona correctamente, podemos entrenar el mismo modelo con los datos con ruido en la entrada y los datos limpios en la salida

In [ ]:
autoencoder.fit(
    x=noisy_train_data,
    y=train_data,
    epochs=100,
    batch_size=128,
    shuffle=True,
    validation_data=(noisy_test_data, test_data),
)

In [ ]:
predictions = autoencoder.predict(noisy_test_data)
display_random(noisy_test_data, predictions)
